In [1]:
import numpy as np
import pandas as pd
import time
import os
import datetime as datetime
from nba_api.stats.endpoints import (
    boxscoreadvancedv3,
    leaguegamefinder,
    boxscoretraditionalv3)
from nba_api.stats.static import teams
import requests
from requests.exceptions import Timeout, ConnectionError

In [4]:
#Scraping future games for the 2025-26 season

# List of months remaining in the 2025-26 season
months = ['february', 'march', 'april']
season_year = 2026
all_games = []

print("Fetching 2025-26 NBA Schedule...")

for month in months:
    url = f"https://www.basketball-reference.com/leagues/NBA_{season_year}_games-{month}.html"
    try:
        # Read the tables from the page
        tables = pd.read_html(url)
        # The first table is usually the schedule
        df = tables[0]
        
        # Rename columns for clarity (Standardizing Headers)
        df.rename(columns={
            'Date': 'Date',
            'Start (ET)': 'Time (ET)',
            'Visitor/Neutral': 'Away Team',
            'Home/Neutral': 'Home Team',
            'Arena': 'Venue'
        }, inplace=True)
        
        # Keep only relevant columns
        cols_to_keep = ['Date', 'Time (ET)', 'Away Team', 'Home Team', 'Venue']
        # Check if columns exist before selecting
        df = df[[c for c in cols_to_keep if c in df.columns]]
        
        # Filter out "Playoffs" headers or empty rows if any exist in the raw data
        df = df[df['Date'] != 'Date']
        
        all_games.append(df)
        print(f"Processed {month.capitalize()}...")
        time.sleep(2) # Be polite to the server
        
    except Exception as e:
        print(f"Could not retrieve data for {month}: {e}")

# Combine all months
full_schedule = pd.concat(all_games, ignore_index=True)

# Filter for future games only (From Jan 27, 2026 onwards)
# Convert date column to datetime objects
full_schedule['Date_Obj'] = pd.to_datetime(full_schedule['Date'])
cutoff_date = pd.Timestamp("2026-01-27")
future_games = full_schedule[full_schedule['Date_Obj'] >= cutoff_date].copy()

# Sort by date
future_games.sort_values(by='Date_Obj', inplace=True)

# Drop helper column
future_games.drop(columns=['Date_Obj'], inplace=True)

# Save to CSV
filename = 'nba_schedule_2025-2026.csv'
future_games.to_csv(filename, index=False)

print(f"Success! Saved {len(future_games)} games to {filename}")

Fetching 2025-26 NBA Schedule...
Processed February...
Processed March...
Processed April...
Success! Saved 501 games to nba_schedule_2025-2026.csv


In [2]:
class NBASeasonScraper:
    def __init__(self, season='2025-26'):
        self.season = season
        self.output_dir = f'{season.replace("-", "-")} season'
        self.teams = teams.get_teams()
        self.delay = 1.0
        self.max_retries = 3
        self.timeout = 60
        
        # Create team ID to name mapping
        self.team_id_to_name = {team['id']: team['full_name'] for team in self.teams}
        
    def create_output_directory(self):
        """Create directory for storing CSV files"""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            print(f"Created directory: {self.output_dir}")
    
    def retry_api_call(self, api_func, *args, **kwargs):
        """Retry an API call with exponential backoff"""
        for attempt in range(self.max_retries):
            try:
                time.sleep(self.delay)
                result = api_func(*args, **kwargs)
                return result
            except (Timeout, ConnectionError) as e:
                wait_time = self.delay * (2 ** attempt)
                print(f"    Timeout (attempt {attempt + 1}/{self.max_retries}). Waiting {wait_time:.1f}s...")
                time.sleep(wait_time)
                if attempt == self.max_retries - 1:
                    print(f"    Failed after {self.max_retries} attempts: {e}")
                    return None
            except Exception as e:
                print(f"    Error: {e}")
                return None
        return None
        
    def get_team_games(self, team_id):
        """Fetch all games for a specific team in the season"""
        print(f"Fetching game list for team ID: {team_id}")
        
        def fetch():
            gamefinder = leaguegamefinder.LeagueGameFinder(
                team_id_nullable=team_id,
                season_nullable=self.season,
                season_type_nullable='Regular Season',
                timeout=self.timeout
            )
            return gamefinder.get_data_frames()[0]
        
        result = self.retry_api_call(fetch)
        return result if result is not None else pd.DataFrame()
    
    def get_playoff_games(self, team_id):
        """Fetch playoff games for a specific team"""
        print(f"Fetching playoff games for team ID: {team_id}")
        
        def fetch():
            gamefinder = leaguegamefinder.LeagueGameFinder(
                team_id_nullable=team_id,
                season_nullable=self.season,
                season_type_nullable='Playoffs',
                timeout=self.timeout
            )
            return gamefinder.get_data_frames()[0]
        
        result = self.retry_api_call(fetch)
        return result if result is not None else pd.DataFrame()
    
    def get_team_box_score_traditional(self, game_id):
        """
        Fetch traditional team box score for a game
        Returns: DataFrame with TEAM-LEVEL stats (not player stats)
        """
        def fetch():
            traditional = boxscoretraditionalv3.BoxScoreTraditionalV3(
                game_id=game_id,
                timeout=self.timeout
            )
            # Index 1 = Team stats (Index 0 = Player stats)
            return traditional.get_data_frames()[1]
        
        result = self.retry_api_call(fetch)
        return result if result is not None else pd.DataFrame()
    
    def get_team_box_score_advanced(self, game_id):
        """
        Fetch advanced team box score for a game
        Returns: DataFrame with TEAM-LEVEL advanced stats
        """
        def fetch():
            advanced = boxscoreadvancedv3.BoxScoreAdvancedV3(
                game_id=game_id,
                timeout=self.timeout
            )
            # Index 1 = Team stats (Index 0 = Player stats)
            return advanced.get_data_frames()[1]
        
        result = self.retry_api_call(fetch)
        return result if result is not None else pd.DataFrame()
    
    def get_team_game_data(self, game_id, team_id):
        """
        Merge traditional and advanced TEAM box score data for a single game
        Only returns data for the specified team (not opponent)
        Also adds opponent team name
        """
        # Fetch TEAM box scores (not player box scores)
        traditional = self.get_team_box_score_traditional(game_id)
        advanced = self.get_team_box_score_advanced(game_id)
        
        # Get opponent name
        opponent_name = "Unknown"
        if traditional is not None and not traditional.empty and 'teamId' in traditional.columns:
            team_ids = traditional['teamId'].tolist()
            for tid in team_ids:
                if tid != team_id:
                    opponent_name = self.team_id_to_name.get(tid, f"Team_{tid}")
                    break
        
        # Filter for specific team and merge
        dataframes = []
        
        if traditional is not None and not traditional.empty:
            if 'teamId' in traditional.columns:
                team_df = traditional[traditional['teamId'] == team_id].copy()
                if not team_df.empty:
                    dataframes.append(team_df)
        
        if advanced is not None and not advanced.empty:
            if 'teamId' in advanced.columns:
                team_df = advanced[advanced['teamId'] == team_id].copy()
                if not team_df.empty:
                    dataframes.append(team_df)
        
        # Merge dataframes
        if len(dataframes) == 2:
            merged = pd.merge(
                dataframes[0], 
                dataframes[1], 
                on=['gameId', 'teamId'], 
                how='outer',
                suffixes=('', '_adv')
            )
            # Remove duplicate columns
            merged = merged.loc[:, ~merged.columns.str.endswith('_adv')]
            # Add opponent name as first column
            merged.insert(0, 'OPPONENT', opponent_name)
            return merged
        elif len(dataframes) == 1:
            result = dataframes[0]
            # Add opponent name as first column
            result.insert(0, 'OPPONENT', opponent_name)
            return result
        
        return pd.DataFrame()
    
    def save_team_data(self, team_name, team_season_df):
        """Save team data to CSV file"""
        filename = f"{self.output_dir}/{team_name.replace(' ', '_')}_2025-26.csv"
        
        # Remove duplicates based on GAME_ID before saving
        if 'GAME_ID' in team_season_df.columns:
            original_count = len(team_season_df)
            team_season_df = team_season_df.drop_duplicates(subset=['GAME_ID'], keep='first')
            deduped_count = len(team_season_df)
            if original_count != deduped_count:
                print(f"    Removed {original_count - deduped_count} duplicate games")
        
        # Sort by game date if available
        if 'GAME_DATE' in team_season_df.columns:
            team_season_df = team_season_df.sort_values('GAME_DATE')
        
        # Save to CSV
        team_season_df.to_csv(filename, index=False)
        return filename
    
    def scrape_team_season(self, team_id, team_name):
        """Scrape entire season for a single team, saving incrementally"""
        print(f"\n{'='*60}")
        print(f"Scraping {team_name} (ID: {team_id})")
        print(f"{'='*60}")
        
        # Define the filename for this team
        filename = f"{self.output_dir}/{team_name.replace(' ', '_')}_2025-26.csv"
        
        # Check if file already exists and load it
        if os.path.exists(filename):
            print(f"Found existing file: {filename}")
            # --- FIX: Added dtype={'GAME_ID': str} to prevent stripping leading zeros ---
            existing_df = pd.read_csv(filename, dtype={'GAME_ID': str})
            # Remove any duplicates from existing file
            if 'GAME_ID' in existing_df.columns:
                existing_df = existing_df.drop_duplicates(subset=['GAME_ID'], keep='first')
            existing_game_ids = set(existing_df['GAME_ID'].values) if 'GAME_ID' in existing_df.columns else set()
            print(f"   Already have {len(existing_game_ids)} games. Will fetch missing games only.")
        else:
            existing_df = pd.DataFrame()
            existing_game_ids = set()
        
        # Get regular season games
        regular_games = self.get_team_games(team_id)
        playoff_games = self.get_playoff_games(team_id)
        
        all_games = pd.concat([regular_games, playoff_games], ignore_index=True)
        
        if all_games.empty:
            print(f"No games found for {team_name}")
            return
        
        # Remove duplicates from API response
        if 'GAME_ID' in all_games.columns:
            original_count = len(all_games)
            all_games = all_games.drop_duplicates(subset=['GAME_ID'], keep='first')
            if original_count != len(all_games):
                print(f"Removed {original_count - len(all_games)} duplicate game entries from API")
        
        print(f"Found {len(all_games)} total games for the season")
        
        # Get unique game IDs
        all_game_ids = set(all_games['GAME_ID'].unique())
        
        # Determine which games need to be fetched
        games_to_fetch = all_game_ids - existing_game_ids
        
        if not games_to_fetch:
            print(f"All games already collected for {team_name}")
            return
        
        print(f"Need to fetch {len(games_to_fetch)} new games")
        
        # Collect detailed stats for each new game
        new_games_list = []
        failed_games = []
        games_fetched = sorted(games_to_fetch)
        
        for idx, game_id in enumerate(games_fetched, 1):
            print(f"[{idx}/{len(games_fetched)}] Game {game_id}", end=" ")
            
            # Get basic game info from leaguegamefinder
            game_info = all_games[all_games['GAME_ID'] == game_id].iloc[0].to_dict()
            
            # Get detailed TEAM box scores (includes opponent name)
            detailed_stats = self.get_team_game_data(game_id, team_id)
            
            if not detailed_stats.empty:
                # Combine basic info with detailed stats
                for key, value in game_info.items():
                    if key not in detailed_stats.columns:
                        detailed_stats[key] = value
                
                new_games_list.append(detailed_stats)
                
                # SAVE EVERY 5 GAMES
                if idx % 5 == 0 or idx == len(games_fetched):
                    if not existing_df.empty:
                        updated_df = pd.concat([existing_df] + new_games_list, ignore_index=True)
                    else:
                        updated_df = pd.concat(new_games_list, ignore_index=True)
                    
                    self.save_team_data(team_name, updated_df)
                    print(f"SAVED ({len(updated_df)} total)")
                else:
                    print(f"OK")
                
            else:
                print(f"FAILED")
                failed_games.append(game_id)
        
        # Final save with all games
        if new_games_list:
            if not existing_df.empty:
                final_df = pd.concat([existing_df] + new_games_list, ignore_index=True)
            else:
                final_df = pd.concat(new_games_list, ignore_index=True)
            
            saved_file = self.save_team_data(team_name, final_df)
            print(f"\nComplete: {len(final_df)} total games | {len(final_df.columns)} columns")
            
            if failed_games:
                print(f"Failed: {len(failed_games)} games - {failed_games[:3]}...")
        elif not existing_df.empty:
            print(f"No new games added")
        else:
            print(f"No data collected")
    
    def scrape_all_teams(self, start_from_team=None, teams_list=None):
        """
        Scrape data for NBA teams
        
        Parameters:
        -----------
        start_from_team : str, optional
            Team name to start from
        teams_list : list, optional
            List of specific team names to scrape
        """
        self.create_output_directory()
        
        print(f"\n{'='*60}")
        print(f"NBA Season Scraper - {self.season}")
        print(f"{'='*60}")
        print(f"Output: {self.output_dir}")
        print(f"Fetching TEAM-LEVEL stats with OPPONENT names")
        print(f"Delay: {self.delay}s | Timeout: {self.timeout}s | Retries: {self.max_retries}")
        print(f"Saves every 5 games with deduplication\n")
        
        # Determine which teams to process
        if teams_list:
            teams_to_process = [t for t in self.teams if t['full_name'] in teams_list]
            print(f"Processing {len(teams_to_process)} specific teams")
        else:
            teams_to_process = self.teams
            print(f"Processing all {len(teams_to_process)} teams")
        
        start_idx = 0
        if start_from_team and not teams_list:
            for idx, team in enumerate(teams_to_process):
                if team['full_name'].lower() == start_from_team.lower():
                    start_idx = idx
                    print(f"Starting from: {team['full_name']}\n")
                    break
        
        for idx, team in enumerate(teams_to_process[start_idx:], start_idx + 1):
            team_id = team['id']
            team_name = team['full_name']
            
            print(f"\n{'='*60}")
            print(f"[{idx}/{len(teams_to_process)}] {team_name}")
            print(f"{'='*60}")
            
            try:
                self.scrape_team_season(team_id, team_name)
            except KeyboardInterrupt:
                print(f"\n\nInterrupted!")
                print(f"Last team: {team_name}")
                print(f"Resume: scraper.scrape_all_teams(start_from_team='{team_name}')")
                raise
            except Exception as e:
                print(f"ERROR: {e}")
                import traceback
                traceback.print_exc()
                continue
            
            # Delay between teams
            if idx < len(teams_to_process):
                print(f"Waiting 2s before next team...")
                time.sleep(2)
        
        print(f"\n{'='*60}")
        print(f"COMPLETE! Files in: {self.output_dir}")
        print(f"{'='*60}")


def main():
    scraper = NBASeasonScraper(season='2025-26')
    
    # Option 1: Scrape all teams
    scraper.scrape_all_teams()
    
    # Option 2: Scrape specific teams only
    # scraper.scrape_all_teams(teams_list=['Los Angeles Lakers', 'Boston Celtics'])
    
    # Option 3: Resume from a specific team
    # scraper.scrape_all_teams(start_from_team='Chicago Bulls')
    
    print("\nDone!")


if __name__ == "__main__":
    main()


NBA Season Scraper - 2025-26
Output: 2025-26 season
Fetching TEAM-LEVEL stats with OPPONENT names
Delay: 1.0s | Timeout: 60s | Retries: 3
Saves every 5 games with deduplication

Processing all 30 teams

[1/30] Atlanta Hawks

Scraping Atlanta Hawks (ID: 1610612737)
Found existing file: 2025-26 season/Atlanta_Hawks_2025-26.csv
   Already have 54 games. Will fetch missing games only.
Fetching game list for team ID: 1610612737
Fetching playoff games for team ID: 1610612737


C:\Users\jeong\AppData\Local\Temp\ipykernel_9104\2647262656.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games = pd.concat([regular_games, playoff_games], ignore_index=True)


Found 55 total games for the season
Need to fetch 1 new games
[1/1] Game 0022500765     Removed 1 duplicate games
SAVED (56 total)
    Removed 1 duplicate games

Complete: 56 total games | 78 columns
Waiting 2s before next team...

[2/30] Boston Celtics

Scraping Boston Celtics (ID: 1610612738)
Found existing file: 2025-26 season/Boston_Celtics_2025-26.csv
   Already have 53 games. Will fetch missing games only.
Fetching game list for team ID: 1610612738
Fetching playoff games for team ID: 1610612738


C:\Users\jeong\AppData\Local\Temp\ipykernel_9104\2647262656.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games = pd.concat([regular_games, playoff_games], ignore_index=True)


Found 53 total games for the season
All games already collected for Boston Celtics
Waiting 2s before next team...

[3/30] Cleveland Cavaliers

Scraping Cleveland Cavaliers (ID: 1610612739)
Found existing file: 2025-26 season/Cleveland_Cavaliers_2025-26.csv
   Already have 53 games. Will fetch missing games only.
Fetching game list for team ID: 1610612739
Fetching playoff games for team ID: 1610612739


...

Found 52 total games for the season
All games already collected for Washington Wizards
Waiting 2s before next team...

[29/30] Detroit Pistons

Scraping Detroit Pistons (ID: 1610612765)
Found existing file: 2025-26 season/Detroit_Pistons_2025-26.csv
   Already have 51 games. Will fetch missing games only.
Fetching game list for team ID: 1610612765
Fetching playoff games for team ID: 1610612765


C:\Users\jeong\AppData\Local\Temp\ipykernel_9104\2647262656.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games = pd.concat([regular_games, playoff_games], ignore_index=True)


Found 52 total games for the season
Need to fetch 1 new games
[1/1] Game 0022500761     Removed 1 duplicate games
SAVED (53 total)
    Removed 1 duplicate games

Complete: 53 total games | 78 columns
Waiting 2s before next team...

[30/30] Charlotte Hornets

Scraping Charlotte Hornets (ID: 1610612766)
Found existing file: 2025-26 season/Charlotte_Hornets_2025-26.csv
   Already have 53 games. Will fetch missing games only.
Fetching game list for team ID: 1610612766
Fetching playoff games for team ID: 1610612766


C:\Users\jeong\AppData\Local\Temp\ipykernel_9104\2647262656.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games = pd.concat([regular_games, playoff_games], ignore_index=True)


Found 54 total games for the season
Need to fetch 1 new games
[1/1] Game 0022500761     Removed 1 duplicate games
SAVED (55 total)
    Removed 1 duplicate games

Complete: 55 total games | 78 columns

COMPLETE! Files in: 2025-26 season

Done!
